# T6. Taller 6: Implementar ejemplos de SimPy 

Diego Fernando Palacios Hoyos

codigo: 160003727



# 1)Ejemplo de renegocia bancaria

Cubiertas

- Recursos: recurso
- Acontecimientos de condición

Guión:
  Un mostrador con un tiempo de servicio aleatorio y clientes que renegan. Basado en el
  programa bank08.py del tutorial de TheBank de SimPy 2. (KGM)



In [6]:
import random

import simpy


RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Numero total de clientes
INTERVAL_CUSTOMERS = 10.0  # Genere nuevos clientes aproximadamente cada x segundos
MIN_PATIENCE = 1  # Min. paciencia del cliente
MAX_PATIENCE = 3  # Max. paciencia del cliente


def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Cliente %02d ' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """El cliente llega, se sirve y se va."""
    arrive = env.now
    print('%7.4f %s: Aquí estoy' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Espera el contador o aborta al final de nuestra cuerda
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # Llegamos al mostrador
            print('%7.4f %s: Esperado %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Terminado' % (env.now, name))

        else:
            # Nosotros renegamos
            print('%7.4f %s: RENEGADO despues de %6.3f' % (env.now, name, wait))


# Configurar e iniciar la simulación.
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Iniciar procesos y ejecutar
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Bank renege
 0.0000 Cliente 00 : Aquí estoy
 0.0000 Cliente 00 : Esperado  0.000
 3.8595 Cliente 00 : Terminado
10.2006 Cliente 01 : Aquí estoy
10.2006 Cliente 01 : Esperado  0.000
12.7265 Cliente 02 : Aquí estoy
13.9003 Cliente 02 : RENEGADO despues de  1.174
23.7507 Cliente 01 : Terminado
34.9993 Cliente 03 : Aquí estoy
34.9993 Cliente 03 : Esperado  0.000
37.9599 Cliente 03 : Terminado
40.4798 Cliente 04 : Aquí estoy
40.4798 Cliente 04 : Esperado  0.000
43.1401 Cliente 04 : Terminado


# 2)Ejemplo de lavado de autos.

Cubiertas

- Esperando otros procesos
- Recursos: recurso

Guión:
  Un lavado de autos tiene un número limitado de lavadoras y define
  Un proceso de lavado que lleva algún tiempo (aleatorio).

  Los procesos del automóvil llegan al lavado de autos en un momento aleatorio. Si un lavado
  la máquina está disponible, comienzan el proceso de lavado y esperan
  para terminar. Si no, esperan hasta que usen uno.

In [12]:
import random

import simpy


RANDOM_SEED = 42
NUM_MACHINES = 2  # Número de máquinas en el lavado de autos
WASHTIME = 5      # Minutos que toma limpiar un auto
T_INTER = 7       # Crea un auto cada ~ 7 minutos
SIM_TIME = 20     # Tiempo de simulación en minutos


class Carwash(object):
    """ Un lavado de autos tiene un número limitado de máquinas (`` NUM_MACHINES``) para
    limpie los autos en paralelo.

    Los autos tienen que solicitar una de las máquinas. Cuando consiguieron uno, ellos
    puede comenzar los procesos de lavado y esperar a que termine (que
    toma `` tiempo de lavado '' minutos).
    """
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car):
        """The washing processes. It takes a ``car`` processes and tries
        to clean it."""
        yield self.env.timeout(WASHTIME)
        print("Lavado de autos elimino  %d%% de suciedad del %s " %
              (random.randint(50, 99), car))


def car(env, name, cw):
    """El proceso del auto (cada auto tiene un `` nombre '') llega al lavado de autos
    (`` cw``) y solicita una máquina de limpieza.

    Luego comienza el proceso de lavado, espera a que termine y
    deja de no volver nunca ...
    """
    print('%s llega al lavado de autos a %.2f.' % (name, env.now))
    with cw.machine.request() as request:
        yield request

        print('%s entra en el lavado de autos a las %.2f.' % (name, env.now))
        yield env.process(cw.wash(name))

        print('%s deja el lavado de autos a las %.2f.' % (name, env.now))


def setup(env, num_machines, washtime, t_inter):
    """Crea un lavado de autos, varios autos iniciales y sigue creando autos
    aprox. cada `` t_inter`` minutos."""
    # Create the carwash
    carwash = Carwash(env, num_machines, washtime)

    # Create 4 initial cars
    for i in range(4):
        env.process(car(env, 'Carro %d' % i, carwash))

    # Create more cars while the simulation is running
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, 'Carro %d' % i, carwash))


# Setup and start the simulation
print('Carwash')
random.seed(RANDOM_SEED)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))

# Execute!
env.run(until=SIM_TIME)

Carwash
Carro 0 llega al lavado de autos a 0.00.
Carro 1 llega al lavado de autos a 0.00.
Carro 2 llega al lavado de autos a 0.00.
Carro 3 llega al lavado de autos a 0.00.
Carro 0 entra en el lavado de autos a las 0.00.
Carro 1 entra en el lavado de autos a las 0.00.
Carro 4 llega al lavado de autos a 5.00.
Lavado de autos elimino  97% de suciedad del Carro 0 
Lavado de autos elimino  67% de suciedad del Carro 1 
Carro 0 deja el lavado de autos a las 5.00.
Carro 1 deja el lavado de autos a las 5.00.
Carro 2 entra en el lavado de autos a las 5.00.
Carro 3 entra en el lavado de autos a las 5.00.
Carro 5 llega al lavado de autos a 10.00.
Lavado de autos elimino  64% de suciedad del Carro 2 
Lavado de autos elimino  58% de suciedad del Carro 3 
Carro 2 deja el lavado de autos a las 10.00.
Carro 3 deja el lavado de autos a las 10.00.
Carro 4 entra en el lavado de autos a las 10.00.
Carro 5 entra en el lavado de autos a las 10.00.
Lavado de autos elimino  97% de suciedad del Carro 4 
Lavado 


# 3)Ejemplo de taller de máquinas

Cubiertas

- interrupciones
- Recursos: recurso preventivo

Guión:
  Un taller tiene * n * máquinas idénticas. Una corriente de trabajos (suficiente para
  mantener las máquinas ocupadas) llega. Cada máquina se descompone
  periódicamente Las reparaciones son realizadas por un reparador. El reparador
  también tiene otras tareas menos importantes que realizar. Máquinas rotas
  evitar estas tareas. El reparador los continúa cuando termina
  Con la reparación de la máquina. El taller trabaja continuamente.

In [17]:
import random

import simpy

RANDOM_SEED = 42
PT_MEAN = 10.0 # Promedio tiempo de procesamiento en minutos
PT_SIGMA = 2.0 # Sigma del tiempo de procesamiento
MTTF = 300.0 # Tiempo medio de falla en minutos
BREAK_MEAN = 1 / MTTF # Param. para distribución expovariada
REPAIR_TIME = 30.0 # Tiempo que lleva reparar una máquina en minutos
JOB_DURATION = 30.0 # Duración de otros trabajos en minutos
NUM_MACHINES = 10 # Número de máquinas en el taller
WEEKS = 4 # Tiempo de simulación en semanas
SIM_TIME = WEEKS * 7 * 24 * 60 # Tiempo de simulación en minutos

def time_per_part():
    """Devuelva el tiempo de procesamiento real para una parte concreta."""
    return random.normalvariate(PT_MEAN, PT_SIGMA)


def time_to_failure():
    """Tiempo de retorno hasta el próximo fallo de una máquina."""
    return random.expovariate(BREAK_MEAN)


class Machine(object):
    """Una máquina produce piezas y me rompo de vez en cuando.

    Si se rompe, solicita un * reparador * y continúa la producción.
    después de que se repare.

    Una máquina tiene un * nombre * y un número de * partes_fabricadas * hasta el momento.
    """
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        self.parts_made = 0
        self.broken = False

        # Start "working" and "break_machine" processes for this machine.
        self.process = env.process(self.working(repairman))
        env.process(self.break_machine())

    def working(self, repairman):
        """Produzca piezas mientras se ejecute la simulación.

        Al hacer una pieza, la máquina puede romperse varias veces.
        Solicite un reparador cuando esto suceda.

        """
        while True:
            # Start making a new part
            done_in = time_per_part()
            while done_in:
                try:
                    # Working on the part
                    start = self.env.now
                    yield self.env.timeout(done_in)
                    done_in = 0  # Set to 0 to exit while loop.

                except simpy.Interrupt:
                    self.broken = True
                    done_in -= self.env.now - start  # How much time left?

                    # Request a repairman. This will preempt its "other_job".
                    with repairman.request(priority=1) as req:
                        yield req
                        yield self.env.timeout(REPAIR_TIME)

                    self.broken = False

            # Part is done.
            self.parts_made += 1

    def break_machine(self):
        """Rompa la máquina de vez en cuando."""
        while True:
            yield self.env.timeout(time_to_failure())
            if not self.broken:
                # Only break the machine if it is currently working.
                self.process.interrupt()


def other_jobs(env, repairman):
    """El otro trabajo del reparador (sin importancia)."""
    while True:
        # Start a new job
        done_in = JOB_DURATION
        while done_in:
            # Retry the job until it is done.
            # It's priority is lower than that of machine repairs.
            with repairman.request(priority=2) as req:
                yield req
                try:
                    start = env.now
                    yield env.timeout(done_in)
                    done_in = 0
                except simpy.Interrupt:
                    done_in -= env.now - start


# Setup and start the simulation
print('Tienda de máquina')
random.seed(RANDOM_SEED)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
repairman = simpy.PreemptiveResource(env, capacity=1)
machines = [Machine(env, 'Máquina %d' % i, repairman)
            for i in range(NUM_MACHINES)]
env.process(other_jobs(env, repairman))

# Execute!
env.run(until=SIM_TIME)

# Analyis/results
print('Resultados del taller de máquinas después %s semanas' % WEEKS)
for machine in machines:
    print('%s hecho %d partes.' % (machine.name, machine.parts_made))

Tienda de máquina
Resultados del taller de máquinas después 4 semanas
Máquina 0 hecho 3251 partes.
Máquina 1 hecho 3273 partes.
Máquina 2 hecho 3242 partes.
Máquina 3 hecho 3343 partes.
Máquina 4 hecho 3387 partes.
Máquina 5 hecho 3244 partes.
Máquina 6 hecho 3269 partes.
Máquina 7 hecho 3185 partes.
Máquina 8 hecho 3302 partes.
Máquina 9 hecho 3279 partes.


# 4)Ejemplo de renege de películas

Cubiertas

- Recursos: recurso
- Acontecimientos de condición
- Eventos compartidos

Guión:
  Un cine tiene un mostrador de venta de boletos para tres
  películas (solo en el próximo programa). Cuando se agota una película, todas las personas esperan
  comprar entradas para esa película renege (dejar la cola).

In [22]:
import collections
import random

import simpy


RANDOM_SEED = 42
TICKETS = 50  # Number of tickets per movie
SIM_TIME = 120  # Simulate until


def moviegoer(env, movie, num_tickets, theater):
    """Un espectador intenta por una cantidad de boletos (* num_tickets *) para
    cierta * película * en un * teatro *.

    Si la película se agota, ella deja el teatro. Si ella consigue
    en el mostrador, ella trata de comprar una serie de boletos. Si no es suficiente
    quedan boletos, discute con el cajero y se va.

    Si queda un boleto como máximo después de que el espectador la haya comprado
    entradas, el evento * vendido * para esta película se dispara causando
    todos los cinéfilos restantes para irse.
    """
    with theater.counter.request() as my_turn:
        # Wait until its our turn or until the movie is sold out
        result = yield my_turn | theater.sold_out[movie]

        # Check if it's our turn of if movie is sold out
        if my_turn not in result:
            theater.num_renegers[movie] += 1
            env.exit()

        # Check if enough tickets left.
        if theater.available[movie] < num_tickets:
            # Moviegoer leaves after some discussion
            yield env.timeout(0.5)
            env.exit()

        # Buy tickets
        theater.available[movie] -= num_tickets
        if theater.available[movie] < 2:
            # Trigger the "sold out" event for the movie
            theater.sold_out[movie].succeed()
            theater.when_sold_out[movie] = env.now
            theater.available[movie] = 0
        yield env.timeout(1)


def customer_arrivals(env, theater):
    """Crea nuevos * espectadores * hasta que el tiempo de simulación llegue a 120."""
    while True:
        yield env.timeout(random.expovariate(1 / 0.5))

        movie = random.choice(theater.movies)
        num_tickets = random.randint(1, 6)
        if theater.available[movie]:
            env.process(moviegoer(env, movie, num_tickets, theater))


Theater = collections.namedtuple('Theater', 'counter, movies, available, '
                                            'sold_out, when_sold_out, '
                                            'num_renegers')


# Setup and start the simulation
print('Peliculas')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Create movie theater
counter = simpy.Resource(env, capacity=1)
movies = ['Python Unchained', 'Kill Process', 'Pulp Implementation']
available = {movie: TICKETS for movie in movies}
sold_out = {movie: env.event() for movie in movies}
when_sold_out = {movie: None for movie in movies}
num_renegers = {movie: 0 for movie in movies}
theater = Theater(counter, movies, available, sold_out, when_sold_out,
                  num_renegers)

# Start process and run
env.process(customer_arrivals(env, theater))
env.run(until=SIM_TIME)

# Analysis/results
for movie in movies:
    if theater.sold_out[movie]:
        print('Peicula "%s" Agotado %.1f minutos después del mostrador '
              'apertura.' % (movie, theater.when_sold_out[movie]))
        print('  Número de personas que dejan la cola cuando se agotaron las películas: %s' %
              theater.num_renegers[movie])

Peliculas
Peicula "Python Unchained" Agotado 38.0 minutos después del mostrador apertura.
  Número de personas que dejan la cola cuando se agotaron las películas: 16
Peicula "Kill Process" Agotado 43.0 minutos después del mostrador apertura.
  Número de personas que dejan la cola cuando se agotaron las películas: 5
Peicula "Pulp Implementation" Agotado 28.0 minutos después del mostrador apertura.
  Número de personas que dejan la cola cuando se agotaron las películas: 5


# 5)Ejemplo de repostaje de gasolinera

Cubiertas

- Recursos: recurso
- Recursos: Contenedor
- Esperando otros procesos

Guión:
  Una estación de servicio tiene un número limitado de bombas de combustible que comparten un común
  depósito de combustible Los autos llegan al azar a la estación de servicio, solicite uno
  de las bombas de combustible y comenzar a repostar desde ese depósito.

  Un proceso de control de la estación de servicio observa el nivel de combustible de la estación de servicio.
  y llama a un camión tanque para repostar si el nivel de la estación baja
  debajo de un umbral

In [27]:
import itertools
import random

import simpy

RANDOM_SEED = 42
GAS_STATION_SIZE = 200 # litros
THRESHOLD = 10 # Umbral para llamar al camión cisterna (en%)
FUEL_TANK_SIZE = 50 # litros
FUEL_TANK_LEVEL = [5, 25] # Niveles mínimo / máximo de tanques de combustible (en litros)
REFUELING_SPEED = 2 # litros / segundo
TANK_TRUCK_TIME = 300 # Segundos tarda el camión tanque en llegar
T_INTER = [30, 300] # Crear un auto cada [min, max] segundos
SIM_TIME = 1000 # Tiempo de simulación en segundos

def car(name, env, gas_station, fuel_pump):
    """Un automóvil llega a la estación de servicio para repostar.

    Solicita una de las bombas de combustible de la estación de servicio e intenta obtener el
    cantidad deseada de gas de ella. Si el embalse de la estación es
    agotado, el automóvil tiene que esperar a que llegue el camión cisterna.
    """
    fuel_tank_level = random.randint(*FUEL_TANK_LEVEL)
    print('%s llegando a la estación de servicio a las %.1f' % (name, env.now))
    with gas_station.request() as req:
        start = env.now
        # Request one of the gas pumps
        yield req

        # Get the required amount of fuel
        liters_required = FUEL_TANK_SIZE - fuel_tank_level
        yield fuel_pump.get(liters_required)

        # The "actual" refueling process takes some time
        yield env.timeout(liters_required / REFUELING_SPEED)

        print('%s terminó de repostar en% .1f segundos.' % (name,
                                                          env.now - start))


def gas_station_control(env, fuel_pump):
    """Verifique periódicamente el nivel de * fuel_pump * y llame al tanque
    camión si el nivel cae por debajo de un umbral."""
    while True:
        if fuel_pump.level / fuel_pump.capacity * 100 < THRESHOLD:
            # We need to call the tank truck now!
            print('Llamar a camión cisterna al %d' % env.now)
            # Wait for the tank truck to arrive and refuel the station
            yield env.process(tank_truck(env, fuel_pump))

        yield env.timeout(10)  # Check every 10 seconds


def tank_truck(env, fuel_pump):
    """Llega a la estación de servicio después de un cierto retraso y la reabastece."""
    yield env.timeout(TANK_TRUCK_TIME)
    print('Camión tanque llegando a tiempo. %d' % env.now)
    ammount = fuel_pump.capacity - fuel_pump.level
    print('Repostaje de camiones cisterna% .1f litros.' % ammount)
    yield fuel_pump.put(ammount)


def car_generator(env, gas_station, fuel_pump):
    """Genere autos nuevos que lleguen a la estación de servicio."""
    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(car('Carro %d' % i, env, gas_station, fuel_pump))


# Setup and start the simulation
print('Repostaje de gasolineras')
random.seed(RANDOM_SEED)

# Create environment and start processes
env = simpy.Environment()
gas_station = simpy.Resource(env, 2)
fuel_pump = simpy.Container(env, GAS_STATION_SIZE, init=GAS_STATION_SIZE)
env.process(gas_station_control(env, fuel_pump))
env.process(car_generator(env, gas_station, fuel_pump))

# Execute!
env.run(until=SIM_TIME)

Repostaje de gasolineras
Carro 0 llegando a la estación de servicio a las 87.0
Carro 0 terminó de repostar en 18.5 segundos.
Carro 1 llegando a la estación de servicio a las 129.0
Carro 1 terminó de repostar en 19.0 segundos.
Carro 2 llegando a la estación de servicio a las 284.0
Carro 2 terminó de repostar en 21.0 segundos.
Carro 3 llegando a la estación de servicio a las 385.0
Carro 3 terminó de repostar en 13.5 segundos.
Carro 4 llegando a la estación de servicio a las 459.0
Llamar a camión cisterna al 460
Carro 4 terminó de repostar en 22.0 segundos.
Carro 5 llegando a la estación de servicio a las 705.0
Carro 6 llegando a la estación de servicio a las 750.0
Camión tanque llegando a tiempo. 760
Repostaje de camiones cisterna 188.0 litros.
Carro 6 terminó de repostar en 29.0 segundos.
Carro 5 terminó de repostar en 76.5 segundos.
Carro 7 llegando a la estación de servicio a las 891.0
Carro 7 terminó de repostar en 13.0 segundos.


# 6) Ejemplo de comunicación de proceso

Cubiertas

- Recursos: Tienda

Guión:
  Este ejemplo muestra cómo interconectar elementos del modelo de simulación
  juntos usando: clase: `~ simpy.resources.store.Store` para uno a uno,
  y procesos asincrónicos de muchos a uno. Para uno a muchos simple
  La clase BroadCastPipe se construye desde la Tienda.

Cuando es útil:
  Cuando un proceso de consumo no siempre espera un proceso generador
  y estos procesos se ejecutan de forma asincrónica. Este ejemplo muestra cómo
  crear un búfer y también decir si el proceso del consumidor se retrasó
  cediendo al evento desde un proceso generador.

  Esto también es útil cuando se necesita transmitir alguna información a
  muchos procesos de recepción

  Finalmente, el uso de tuberías puede simplificar la forma en que los procesos están interconectados
  entre sí en un modelo de simulación.


In [31]:
import random

import simpy


RANDOM_SEED = 42
SIM_TIME = 100


class BroadcastPipe(object):
    """Una tubería de difusión que permite que un proceso envíe mensajes a muchos.

    Esta construcción es útil cuando los consumidores de mensajes se están ejecutando en
    diferentes tasas que los generadores de mensajes y proporciona un evento
    amortiguamiento a los procesos de consumo.

    Los parámetros se utilizan para crear un nuevo
    : clase: `~ instancia simpy.resources.store.Store` cada vez
    : meth: se llama a `get_output_conn ()`.

    """
    def __init__(self, env, capacity=simpy.core.Infinity):
        self.env = env
        self.capacity = capacity
        self.pipes = []

    def put(self, value):
        """Transmitir un * valor * a todos los receptores."""
        if not self.pipes:
            raise RuntimeError('There are no output pipes.')
        events = [store.put(value) for store in self.pipes]
        return self.env.all_of(events)  # Condition event for all "events"

    def get_output_conn(self):
        """Obtenga una nueva conexión de salida para este canal de transmisión.

        El valor de retorno es a: class: `~ simpy.resources.store.Store`.
        """
        pipe = simpy.Store(self.env, capacity=self.capacity)
        self.pipes.append(pipe)
        return pipe


def message_generator(name, env, out_pipe):
    """Un proceso que genera mensajes al azar."""
    while True:
        # wait for next transmission
        yield env.timeout(random.randint(6, 10))

        # messages are time stamped to later check if the consumer was
        # late getting them.  Note, using event.triggered to do this may
        # result in failure due to FIFO nature of simulation yields.
        # (i.e. if at the same env.now, message_generator puts a message
        # in the pipe first and then message_consumer gets from pipe,
        # the event.triggered will be True in the other order it will be
        # False
        msg = (env.now, '%s saluda a las %d' % (name, env.now))
        out_pipe.put(msg)


def message_consumer(name, env, in_pipe):
    """Un proceso que consume mensajes."""
    while True:
        # Get event for message pipe
        msg = yield in_pipe.get()

        if msg[0] < env.now:
            # if message was already put into pipe, then
            # message_consumer was late getting to it. Depending on what
            # is being modeled this, may, or may not have some
            # significance
            print('Mensaje TARDE: a la hora %d: %s mensaje recibido: %s' %
                  (env.now, name, msg[1]))

        else:
            # message_consumer is synchronized with message_generator
            print('en el momento %d: %s mensaje recibido: %s.' %
                  (env.now, name, msg[1]))

        # Process does some other work, which may result in missing messages
        yield env.timeout(random.randint(4, 8))


# Setup and start the simulation
print('Comunicación de proceso')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# For one-to-one or many-to-one type pipes, use Store
pipe = simpy.Store(env)
env.process(message_generator('Generador A', env, pipe))
env.process(message_consumer('Consumidor A', env, pipe))

print('\nOne-to-one pipe communication\n')
env.run(until=SIM_TIME)

# For one-to many use BroadcastPipe
# (Note: could also be used for one-to-one,many-to-one or many-to-many)
env = simpy.Environment()
bc_pipe = BroadcastPipe(env)

env.process(message_generator('Generator A', env, bc_pipe))
env.process(message_consumer('Consumidor A', env, bc_pipe.get_output_conn()))
env.process(message_consumer('Consumidor B', env, bc_pipe.get_output_conn()))

print('\nOne-to-many pipe communication\n')
env.run(until=SIM_TIME)

Comunicación de proceso

One-to-one pipe communication

en el momento 6: Consumidor A mensaje recibido: Generador A saluda a las 6.
en el momento 12: Consumidor A mensaje recibido: Generador A saluda a las 12.
en el momento 19: Consumidor A mensaje recibido: Generador A saluda a las 19.
en el momento 26: Consumidor A mensaje recibido: Generador A saluda a las 26.
en el momento 36: Consumidor A mensaje recibido: Generador A saluda a las 36.
en el momento 46: Consumidor A mensaje recibido: Generador A saluda a las 46.
en el momento 52: Consumidor A mensaje recibido: Generador A saluda a las 52.
en el momento 58: Consumidor A mensaje recibido: Generador A saluda a las 58.
Mensaje TARDE: a la hora 66: Consumidor A mensaje recibido: Generador A saluda a las 65
en el momento 75: Consumidor A mensaje recibido: Generador A saluda a las 75.
en el momento 85: Consumidor A mensaje recibido: Generador A saluda a las 85.
en el momento 95: Consumidor A mensaje recibido: Generador A saluda a las 95.


# 7)Ejemplo de latencia de evento

Cubiertas

- Recursos: Tienda

Guión:
  Este ejemplo muestra cómo separar el tiempo de retraso de los eventos entre
  procesos de los procesos mismos.

Cuando es útil:
  Al modelar cosas físicas como cables, propagación de RF, etc.
  mejor encapsulación para mantener este mecanismo de propagación fuera del
  Procesos de envío y recepción.

  También se puede usar para interconectar procesos que envían mensajes

In [36]:
import simpy


SIM_DURATION = 100


class Cable(object):
    """Esta clase representa la propagación a través de un cable.."""
    def __init__(self, env, delay):
        self.env = env
        self.delay = delay
        self.store = simpy.Store(env)

    def latency(self, value):
        yield self.env.timeout(self.delay)
        self.store.put(value)

    def put(self, value):
        self.env.process(self.latency(value))

    def get(self):
        return self.store.get()


def sender(env, cable):
    """Un proceso que genera mensajes al azar.."""
    while True:
        # wait for next transmission
        yield env.timeout(5)
        cable.put('el remitente envió esto a las %d' % env.now)


def receiver(env, cable):
    """Un proceso que consume mensajes.."""
    while True:
        # Get event for message pipe
        msg = yield cable.get()
        print('Recibí esto en %d mientras %s' % (env.now, msg))


# Setup and start the simulation
print('Latencia de eventos')
env = simpy.Environment()

cable = Cable(env, 10)
env.process(sender(env, cable))
env.process(receiver(env, cable))

env.run(until=SIM_DURATION)

Latencia de eventos
Recibí esto en 15 mientras el remitente envió esto a las 5
Recibí esto en 20 mientras el remitente envió esto a las 10
Recibí esto en 25 mientras el remitente envió esto a las 15
Recibí esto en 30 mientras el remitente envió esto a las 20
Recibí esto en 35 mientras el remitente envió esto a las 25
Recibí esto en 40 mientras el remitente envió esto a las 30
Recibí esto en 45 mientras el remitente envió esto a las 35
Recibí esto en 50 mientras el remitente envió esto a las 40
Recibí esto en 55 mientras el remitente envió esto a las 45
Recibí esto en 60 mientras el remitente envió esto a las 50
Recibí esto en 65 mientras el remitente envió esto a las 55
Recibí esto en 70 mientras el remitente envió esto a las 60
Recibí esto en 75 mientras el remitente envió esto a las 65
Recibí esto en 80 mientras el remitente envió esto a las 70
Recibí esto en 85 mientras el remitente envió esto a las 75
Recibí esto en 90 mientras el remitente envió esto a las 80
Recibí esto en 95 mie